# Setup

See:
https://github.com/karpathy/arxiv-sanity-preserver

Package does much we need, 
but we would lack internal understanding,
until we break it down.

In [1]:
import arxiv
import json
import pandas as pd

from os import listdir
from os.path import isfile, join

In [2]:
# Paths
DATA_PATH = '../data/'
OUTPUT_PATH = '../data/raw/'
CREDS_PATH = '../collection/credentials/'

# Basic search
IMO use search below, this seems to artificially truncate

In [4]:
QUERY = "deepfake"
N =  2000

In [5]:
# sort_by : relevance, lastUpdatedDate, or submittedDate
# max_results : large limit

search = arxiv.Search(
  query = QUERY,
  max_results = N, 
  sort_by = arxiv.SortCriterion.SubmittedDate 
)


In [6]:
for key in search.__dict__.keys():
      print(key)

query
id_list
max_results
sort_by
sort_order


In [7]:
# Quick results check, can still be long to run with many results

for result in search.results():
  print(result.title + '\n')

Adaptive Frequency Learning in Two-branch Face Forgery Detection

Fusing Global and Local Features for Generalized AI-Synthesized Image Detection

Self-supervised Learning of Adversarial Example: Towards Good Generalizations for Deepfake Detection

A New Approach to Improve Learning-based Deepfake Detection in Realistic Conditions

A Novel Framework for Assessment of Learning-based Detectors in Realistic Conditions with Application to Deepfake Detection

Making DeepFakes more spurious: evading deep face forgery detection via trace removal attack

Deepfake Style Transfer Mixture: a First Forensic Ballistics Study on Synthetic Images

Transferable Class-Modelling for Decentralized Source Attribution of GAN-Generated Images

Fairness Evaluation in Deepfake Detection Models using Metamorphic Testing

An Audio-Visual Attention Based Multimodal Network for Fake Talking Face Videos Detection

Voice-Face Homogeneity Tells Deepfake

Towards Benchmarking and Evaluating Deepfake Detection

The Vi

In [8]:
# get all data, not recommended

for result in search.results():     
      for key, value in result._raw.items(): #__dict__.items(): ### this is usual way, but here dict has raw
            print(key)
            print(value)
            print('\n')

id
http://arxiv.org/abs/2203.14315v1


guidislink
True


link
http://arxiv.org/abs/2203.14315v1


updated
2022-03-27T14:25:52Z


updated_parsed
time.struct_time(tm_year=2022, tm_mon=3, tm_mday=27, tm_hour=14, tm_min=25, tm_sec=52, tm_wday=6, tm_yday=86, tm_isdst=0)


published
2022-03-27T14:25:52Z


published_parsed
time.struct_time(tm_year=2022, tm_mon=3, tm_mday=27, tm_hour=14, tm_min=25, tm_sec=52, tm_wday=6, tm_yday=86, tm_isdst=0)


title
Adaptive Frequency Learning in Two-branch Face Forgery Detection


title_detail
{'type': 'text/plain', 'language': None, 'base': 'http://export.arxiv.org/api/query?search_query=deepfake&id_list=&sortBy=submittedDate&sortOrder=descending&start=0&max_results=100', 'value': 'Adaptive Frequency Learning in Two-branch Face Forgery Detection'}


summary
Face forgery has attracted increasing attention in recent applications of
computer vision. Existing detection techniques using the two-branch framework
benefit a lot from a frequency perspective, yet ar

KeyboardInterrupt: 

In [9]:
# Save data to 

# overwrite 'w' or append 'a'
action = 'a'

#save as 
save_as = 'arxiv_' + QUERY

for result in search.results():
      with open (OUTPUT_PATH + save_as + '.jsonl', action) as f:
            json.dump(result._raw, f, default=str) # use raw as __dict__ has raw in it, thus more data
            f.write('\n')  

### Client search (for larger searches)

In [18]:
friendly_client = arxiv.Client(
  page_size = 1000, # 2000 max
  delay_seconds = 10, #3 min
  num_retries = 5
)

action = 'a'

#save as 
save_as = 'arxiv_' + QUERY

# Prints 1000 titles before needing to make another request.
for result in friendly_client.results(arxiv.Search(query=QUERY, sort_by = arxiv.SortCriterion.SubmittedDate)):
      with open (OUTPUT_PATH + save_as + '.jsonl', action) as f:
            json.dump(result._raw, f, default=str) # use raw as __dict__ has raw in it, thus more data
            f.write('\n')

Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done
Sweep done

# Start here for loading existing data

In [19]:
# Check files in data folder
datafiles = [f for f in listdir(OUTPUT_PATH) if isfile(join(OUTPUT_PATH, f))]
print(datafiles)

['arxiv_disinformation.jsonl', 'arxiv_deepfake.jsonl']


In [20]:
# Load df
load_file = datafiles[1]

df = pd.read_json(OUTPUT_PATH + load_file, convert_dates=True, lines=True, orient='records')

In [36]:
df.columns

Index(['id', 'guidislink', 'link', 'updated', 'updated_parsed', 'published',
       'published_parsed', 'title', 'title_detail', 'summary',
       'summary_detail', 'authors', 'author_detail', 'author', 'arxiv_comment',
       'links', 'arxiv_primary_category', 'tags', 'arxiv_affiliation',
       'arxiv_journal_ref', 'arxiv_doi'],
      dtype='object')

In [35]:
df.tail(2)

,id,guidislink,link,updated,updated_parsed,published,published_parsed,title,title_detail,summary,...,authors,author_detail,author,arxiv_comment,links,arxiv_primary_category,tags,arxiv_affiliation,arxiv_journal_ref,arxiv_doi
436,http://arxiv.org/abs/1809.00888v1,True,http://arxiv.org/abs/1809.00888v1,2018-09-04T10:59:22Z,"[2018, 9, 4, 10, 59, 22, 1, 247, 0]",2018-09-04T10:59:22Z,"[2018, 9, 4, 10, 59, 22, 1, 247, 0]",MesoNet: a Compact Facial Video Forgery Detect...,"{'type': 'text/plain', 'language': None, 'base...",This paper presents a method to automatically ...,...,"[{'name': 'Darius Afchar'}, {'name': 'Vincent ...",{'name': 'Isao Echizen'},Isao Echizen,accepted to WIFS 2018,"[{'title': 'doi', 'href': 'http://dx.doi.org/1...","{'term': 'cs.CV', 'scheme': 'http://arxiv.org/...","[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",NaN,NaN,10.1109/WIFS.2018.8630761
437,http://arxiv.org/abs/1806.02877v2,True,http://arxiv.org/abs/1806.02877v2,2018-06-11T19:28:49Z,"[2018, 6, 11, 19, 28, 49, 0, 162, 0]",2018-06-07T19:36:09Z,"[2018, 6, 7, 19, 36, 9, 3, 158, 0]",In Ictu Oculi: Exposing AI Generated Fake Face...,"{'type': 'text/plain', 'language': None, 'base...",The new developments in deep generative networ...,...,"[{'name': 'Yuezun Li'}, {'name': 'Ming-Ching C...",{'name': 'Siwei Lyu'},Siwei Lyu,NaN,"[{'href': 'http://arxiv.org/abs/1806.02877v2',...","{'term': 'cs.CV', 'scheme': 'http://arxiv.org/...","[{'term': 'cs.CV', 'scheme': 'http://arxiv.org...",NaN,NaN,NaN
